In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("rend_2016a2022_modificado.csv",encoding='latin1')

In [4]:
df.isnull().any()

Idalumno        False
Cod.Asign       False
Asignatura      False
Cod.Car.Sec     False
Semestre        False
Convocatoria    False
Anho            False
Ciclo           False
Aprobado        False
Firma.Anho      False
1P              False
2P              False
3P              False
4P               True
4P_60            True
T               False
Asistencia      False
Requisito       False
1R              False
2R              False
Nota.Final       True
periodo1         True
periodo2         True
periodo3         True
P1F             False
P2F             False
P3F             False
dtype: bool

In [5]:
df.drop(columns=['Aprobado','3P', '4P', '4P_60','1R','2R','Nota.Final','periodo1','periodo2','periodo3','P1F','P2F','P3F','Asistencia','Requisito'],inplace=True)

In [6]:
df.head()

,Idalumno,Cod.Asign,Asignatura,Cod.Car.Sec,Semestre,Convocatoria,Anho,Ciclo,Firma.Anho,1P,2P,T
0,1,13224,INGENIERIA ECONOMICA,IND-PLS13,8,1,2022,2,2022,11,29,10
1,1,13013,CALCULO 3,IND-PLS13,3,2,2022,2,0,0,0,0
2,1,13014,PROBABILIDAD Y ESTADISTICA,IND-PLS13,3,2,2022,2,0,0,10,7
3,1,13203,TERMODINAMICA,IND-PLS13,5,2,2022,2,0,6,0,1
4,2,13419,SISTEMAS DE CONTROL,ECA-PLS13,6,1,2022,2,2022,19,31,10


In [7]:
df = df.loc[df['Anho']!=2020]

In [8]:
df["Abandono"] = (df["1P"]>=0)&(df['2P']==0)

In [9]:
df["Abandono"] = df["Abandono"].astype(int)

In [10]:
df

,Idalumno,Cod.Asign,Asignatura,Cod.Car.Sec,Semestre,Convocatoria,Anho,Ciclo,Firma.Anho,1P,2P,T,Abandono
0,1,13224,INGENIERIA ECONOMICA,IND-PLS13,8,1,2022,2,2022,11,29,10,0
1,1,13013,CALCULO 3,IND-PLS13,3,2,2022,2,0,0,0,0,1
2,1,13014,PROBABILIDAD Y ESTADISTICA,IND-PLS13,3,2,2022,2,0,0,10,7,0
3,1,13203,TERMODINAMICA,IND-PLS13,5,2,2022,2,0,6,0,1,1
4,2,13419,SISTEMAS DE CONTROL,ECA-PLS13,6,1,2022,2,2022,19,31,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175434,4482,13229,ADMINISTRACION DE RR HH,IND-PLS13,10,1,2019,1,2019,17,34,10,0
175435,4482,2930,SIMULACION DE SISTEMAS PRODUCTIVOS,INT9-PROYT,13,1,2019,1,2019,22,36,10,0
175436,4483,13223,TECNOLOGIA DE LA INFORMACION,IND-PLS13,8,1,2019,1,2019,23,30,10,0
175437,4483,13229,ADMINISTRACION DE RR HH,IND-PLS13,10,1,2019,1,2019,16,36,10,0


In [11]:
counts_asignatura = df.groupby('Asignatura')['Abandono'].sum().reset_index()
counts_asignatura.sort_values(by='Abandono',ascending=False,inplace=True)


In [12]:
counts_asignatura

,Asignatura,Abandono
276,PASANTIA,1989
48,COMPUTACION,1967
290,PROBABILIDAD Y ESTADISTICA,1298
126,FISICA 4,1258
248,MECANICA DE MATERIALES 1,1249
...,...,...
67,DINAMICA Y CONTROL DE SISTEMAS ELECTRICO DE PO...,0
186,INSTR. PARA CONTROL Y AUTOM.,0
154,GESTION DE SALUD Y SEGURIDAD INDUST.,0
73,DISEÑO DE SIST. DE MANUF.,0


In [13]:
alumno_features = df.groupby(['Idalumno','Anho', 'Ciclo'])['Asignatura','Semestre'].nunique().reset_index()

<ipython-input-13-2e46c3c9969b>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  alumno_features = df.groupby(['Idalumno','Anho', 'Ciclo'])['Asignatura','Semestre'].nunique().reset_index()


In [14]:
alumno_features.head()

,Idalumno,Anho,Ciclo,Asignatura,Semestre
0,1,2022,2,4,3
1,2,2022,2,5,5
2,3,2022,2,6,4
3,4,2022,1,6,4
4,5,2022,2,5,3


In [15]:
alumno_features['Regular'] = alumno_features['Semestre'].apply(lambda x: 0 if x > 1 else 1)
alumno_features.drop(['Semestre'],axis=1,inplace=True)

In [16]:
alumno_dif_semestres = df.groupby(['Idalumno','Anho', 'Ciclo'])['Semestre'].unique().reset_index()

In [17]:
alumno_dif_semestres['DiferenciaSemestres'] = alumno_dif_semestres['Semestre'].apply(lambda x: max(x) - min(x) if max(x) != 13 else (sorted(x, reverse=True)+[0])[1] - min(x))

In [18]:
alumno_dif_semestres

,Idalumno,Anho,Ciclo,Semestre,DiferenciaSemestres
0,1,2022,2,"[8, 3, 5]",5
1,2,2022,2,"[6, 10, 4, 5, 7]",6
2,3,2022,2,"[8, 6, 7, 13]",2
3,4,2022,1,"[6, 7, 9, 10]",4
4,5,2022,2,"[8, 5, 7]",3
...,...,...,...,...,...
29466,4480,2016,1,"[6, 8]",2
29467,4481,2016,1,[2],0
29468,4481,2017,1,[2],0
29469,4482,2019,1,"[8, 10, 13]",2


In [19]:
cant_carreras = df.groupby(['Idalumno','Anho', 'Ciclo'])['Cod.Car.Sec'].nunique().reset_index(name='Cant_Carreras')

In [20]:
cant_carreras

,Idalumno,Anho,Ciclo,Cant_Carreras
0,1,2022,2,1
1,2,2022,2,2
2,3,2022,2,3
3,4,2022,1,1
4,5,2022,2,1
...,...,...,...,...
29466,4480,2016,1,2
29467,4481,2016,1,1
29468,4481,2017,1,1
29469,4482,2019,1,2


In [21]:
cant_matriculaciones_materia = df.groupby(['Idalumno','Asignatura']).size().reset_index(name='veces_matriculado')

In [22]:
cant_matriculaciones_materia.sort_values(by='veces_matriculado',ascending=False)

,Idalumno,Asignatura,veces_matriculado
15413,698,CALCULO 4,10
13433,634,ESTATICA,10
32387,1262,FISICA 3,10
764,76,MECANICA DE MATERIALES 1,10
72225,2660,ESTATICA,10
...,...,...,...
41900,1565,CENTRALES ELECTRICAS,1
41898,1564,QUIMICA GENERAL,1
41895,1564,MATERIALES DE OBRAS CIVILES 1,1
41893,1564,GEOLOGIA GENERAL,1


In [23]:
df = df.merge(cant_matriculaciones_materia,on=['Idalumno','Asignatura'],how='outer')

In [24]:
df['Recursante'] = df['veces_matriculado'].apply(lambda x: 1 if x>1 else 0)

In [25]:
alumno_features['DiferenciaSemestres'] = alumno_dif_semestres['Semestre'].apply(lambda x: max(x) - min(x) if max(x) != 13 else (sorted(x, reverse=True) + [0])[1] - min(x))

In [26]:
alumno_features.rename(columns={'Asignatura':'Cant_Materias'},inplace=True)

In [27]:
alumno_features

,Idalumno,Anho,Ciclo,Cant_Materias,Regular,DiferenciaSemestres
0,1,2022,2,4,0,5
1,2,2022,2,5,0,6
2,3,2022,2,6,0,2
3,4,2022,1,6,0,4
4,5,2022,2,5,0,3
...,...,...,...,...,...,...
29466,4480,2016,1,4,0,2
29467,4481,2016,1,4,1,0
29468,4481,2017,1,4,1,0
29469,4482,2019,1,3,0,2


In [28]:
df = pd.merge(df,cant_carreras,on=['Idalumno','Anho','Ciclo'],how='outer')

In [29]:
df = pd.merge(df,alumno_features,on=['Idalumno','Anho','Ciclo'],how='outer')

In [30]:
df['Firma'] = df['Firma.Anho'].apply(lambda x: 1 if x > 0 else 0)
#df.rename(columns={'Asignatura_x':'Asignatura','Asignatura_y':'Cant_Materias'},inplace=True)

In [31]:
df.head(10)

,Idalumno,Cod.Asign,Asignatura,Cod.Car.Sec,Semestre,Convocatoria,Anho,Ciclo,Firma.Anho,1P,2P,T,Abandono,veces_matriculado,Recursante,Cant_Carreras,Cant_Materias,Regular,DiferenciaSemestres,Firma
0,1,13224,INGENIERIA ECONOMICA,IND-PLS13,8,1,2022,2,2022,11,29,10,0,1,0,1,4,0,5,1
1,1,13013,CALCULO 3,IND-PLS13,3,2,2022,2,0,0,0,0,1,1,0,1,4,0,5,0
2,1,13014,PROBABILIDAD Y ESTADISTICA,IND-PLS13,3,2,2022,2,0,0,10,7,0,1,0,1,4,0,5,0
3,1,13203,TERMODINAMICA,IND-PLS13,5,2,2022,2,0,6,0,1,1,1,0,1,4,0,5,0
4,2,13419,SISTEMAS DE CONTROL,ECA-PLS13,6,1,2022,2,2022,19,31,10,0,1,0,2,5,0,6,1
5,2,13735,REDES INDUSTRIALES,MCT-PLS13,10,1,2022,2,2022,24,32,10,0,1,0,2,5,0,6,1
6,2,13403,FISICA DE SEMICONDUCTORES,ECA-PLS13,4,2,2022,2,2022,17,14,8,0,1,0,2,5,0,6,1
7,2,13413,SISTEMAS DE REDES 2,ECA-PLS13,5,2,2022,2,2022,14,15,8,0,1,0,2,5,0,6,1
8,2,13721,SISTEMAS HIDRAULICOS Y NEUMATICOS,MCT-PLS13,7,2,2022,2,2022,10,35,10,0,1,0,2,5,0,6,1
9,3,13121,HIDROTECNIA 1,CIV-PLS13,8,1,2022,2,2022,11,25,10,0,1,0,3,6,0,2,1


In [32]:
database = df.loc[df['Asignatura'] == 'MECANICA DE MATERIALES 1'].reset_index()

In [33]:
database.columns

Index(['index', 'Idalumno', 'Cod.Asign', 'Asignatura', 'Cod.Car.Sec',
       'Semestre', 'Convocatoria', 'Anho', 'Ciclo', 'Firma.Anho', '1P', '2P',
       'T', 'Abandono', 'veces_matriculado', 'Recursante', 'Cant_Carreras',
       'Cant_Materias', 'Regular', 'DiferenciaSemestres', 'Firma'],
      dtype='object')

In [34]:
database = database.drop(columns=['index','Cod.Asign','Asignatura','Semestre','Convocatoria','Firma.Anho','2P','T','veces_matriculado',])

In [35]:
database

,Idalumno,Cod.Car.Sec,Anho,Ciclo,1P,Abandono,Recursante,Cant_Carreras,Cant_Materias,Regular,DiferenciaSemestres,Firma
0,12,CIV-PLS13,2018,2,3,0,0,4,6,0,3,0
1,23,ELE-PLS13,2016,1,11,0,0,1,6,0,2,1
2,46,ELE-PLS09,2016,1,9,0,0,1,5,0,6,0
3,49,CIV-PLS13,2017,1,21,0,0,1,2,1,0,1
4,50,MEC-PLS09,2016,1,5,0,1,2,5,0,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3635,4469,CIV-PLS13,2018,1,11,0,1,1,3,0,2,1
3636,4469,IND-PLS13,2018,2,10,0,1,2,3,0,1,1
3637,4469,IND-PLS09,2017,1,10,0,1,3,8,0,2,1
3638,4470,IND-PLS13,2022,2,8,0,0,1,4,0,1,0


In [36]:
database.to_csv('database.csv',index=False)